In [1]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

# Insert path to mavenn beginning of path
import os
import sys
# Load mavenn
import sys
path_to_mavenn = '/Users/tareen/Desktop/Research_Projects/2020_mavenn_github/mavenn_git_ssh/'
sys.path.insert(0, path_to_mavenn)

# Load mavenn
import mavenn
print(mavenn.__path__)

['/Users/tareen/Desktop/Research_Projects/2020_mavenn_github/mavenn_git_ssh/mavenn']


In [2]:
# Download mpsa dataset from master branch
url = 'https://github.com/jbkinney/mavenn/blob/master/mavenn/examples/datasets/mpsa_data.csv.gz?raw=true'

data_df = pd.read_csv(url,  
                      compression='gzip',
                      index_col=[0])

data_df = data_df.reset_index().copy()

# Separate test from data_df
ix_test = data_df['set']=='test'
test_df = data_df[ix_test].reset_index(drop=True)
print(f'test N: {len(test_df):,}')

# Remove test data from data_df
data_df = data_df[~ix_test].reset_index(drop=True)
print(f'training + validation N: {len(data_df):,}')
data_df.head(10)

test N: 6,249
training + validation N: 24,234


,set,tot_ct,ex_ct,y,x
0,training,28,2,0.023406,GGAGUGAUG
1,training,315,7,-0.587914,AGUGUGCAA
2,validation,27,0,-0.438475,UAAGCUUUU
3,training,130,2,-0.631467,AUGGUCGGG
4,training,552,19,-0.433012,AGGGCAGGA
5,training,151,184,1.094011,AAGGUCCGC
6,training,88,4,-0.241737,CAGGCAAAC
7,training,105,5,-0.238471,GUGGCAAAU
8,validation,14,0,-0.167408,CAUGCAGCU
9,training,170,14,-0.048222,CAAGUCACU


In [3]:
# Get sequence length
L = len(data_df['x'][0])

# Define model
model = mavenn.Model(L=L,
                     alphabet='rna',
                     gpmap_type='pairwise', 
                     regression_type='GE',
                     ge_noise_model_type='SkewedT',
                     ge_heteroskedasticity_order=2)

In [4]:
# Set training data
model.set_data(x=data_df['x'],
               y=data_df['y'],
               validation_flags=(data_df['set']=='validation'),
               shuffle=True)

N = 24,234 observations set as training data.
Using 24.8% for validation.
Data shuffled.
Time to set data: 0.405 sec.


In [ ]:
# Fit model to data
model.fit(learning_rate=.001,
          epochs=1000,
          batch_size=50,
          early_stopping=True,
          early_stopping_patience=30,
          linear_initialization=False)

Epoch 1/1000


Using TensorFlow backend.
Exception ignored in: <bound method tqdm.__del__ of <tqdm.auto.tqdm object at 0x7fd2056a3128>>
Traceback (most recent call last):
  File "/Users/tareen/miniconda3/lib/python3.6/site-packages/tqdm/std.py", line 1147, in __del__
    self.close()
  File "/Users/tareen/miniconda3/lib/python3.6/site-packages/tqdm/notebook.py", line 289, in close
    self.disp(bar_style='success', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


365/365 [==============================] - 4s 5ms/step - loss: 72.3799 - I_var: -0.4354 - val_loss: 56.2864 - val_I_var: -0.2090
Epoch 2/1000
365/365 [==============================] - 1s 2ms/step - loss: 53.8801 - I_var: -0.0894 - val_loss: 48.0043 - val_I_var: -0.0053
Epoch 3/1000
365/365 [==============================] - 1s 2ms/step - loss: 47.6181 - I_var: 0.0404 - val_loss: 44.2257 - val_I_var: 0.0923
Epoch 4/1000
365/365 [==============================] - 1s 2ms/step - loss: 43.5306 - I_var: 0.1116 - val_loss: 42.3154 - val_I_var: 0.1459
Epoch 5/1000
365/365 [==============================] - 1s 2ms/step - loss: 42.2815 - I_var: 0.1519 - val_loss: 41.5899 - val_I_var: 0.1675
Epoch 6/1000
365/365 [==============================] - 1s 2ms/step - loss: 42.0501 - I_var: 0.1708 - val_loss: 41.0271 - val_I_var: 0.1847
Epoch 7/1000
365/365 [==============================] - 1s 2ms/step - loss: 40.6740 - I_var: 0.1776 - val_loss: 40.8900 - val_I_var: 0.1887
Epoch 8/1000
365/365 [=======

365/365 [==============================] - 1s 3ms/step - loss: 36.8204 - I_var: 0.3224 - val_loss: 36.4100 - val_I_var: 0.3307
Epoch 60/1000
365/365 [==============================] - 1s 3ms/step - loss: 36.7586 - I_var: 0.3255 - val_loss: 36.4411 - val_I_var: 0.3294
Epoch 61/1000
365/365 [==============================] - 1s 3ms/step - loss: 36.5766 - I_var: 0.3227 - val_loss: 36.2254 - val_I_var: 0.3358
Epoch 62/1000
365/365 [==============================] - 1s 3ms/step - loss: 37.2880 - I_var: 0.3219 - val_loss: 36.8458 - val_I_var: 0.3171
Epoch 63/1000
365/365 [==============================] - 1s 3ms/step - loss: 36.4699 - I_var: 0.3227 - val_loss: 36.3365 - val_I_var: 0.3322
Epoch 64/1000
365/365 [==============================] - 1s 3ms/step - loss: 36.1706 - I_var: 0.3240 - val_loss: 36.2433 - val_I_var: 0.3350
Epoch 65/1000
365/365 [==============================] - 1s 3ms/step - loss: 35.9719 - I_var: 0.3232 - val_loss: 36.4477 - val_I_var: 0.3286
Epoch 66/1000
365/365 [====

365/365 [==============================] - 1s 2ms/step - loss: 36.4462 - I_var: 0.3258 - val_loss: 36.5860 - val_I_var: 0.3215
Epoch 118/1000
365/365 [==============================] - 1s 2ms/step - loss: 35.8458 - I_var: 0.3273 - val_loss: 36.0404 - val_I_var: 0.3370
Epoch 119/1000
365/365 [==============================] - 1s 2ms/step - loss: 36.8993 - I_var: 0.3296 - val_loss: 36.4258 - val_I_var: 0.3266
Epoch 120/1000
365/365 [==============================] - 1s 2ms/step - loss: 36.1824 - I_var: 0.3282 - val_loss: 36.0687 - val_I_var: 0.3365
Epoch 121/1000
365/365 [==============================] - 1s 2ms/step - loss: 36.7047 - I_var: 0.3293 - val_loss: 36.3459 - val_I_var: 0.3283
Epoch 122/1000
365/365 [==============================] - 1s 2ms/step - loss: 36.2645 - I_var: 0.3254 - val_loss: 36.0495 - val_I_var: 0.3371
Epoch 123/1000
365/365 [==============================] - 1s 2ms/step - loss: 36.2899 - I_var: 0.3280 - val_loss: 36.4277 - val_I_var: 0.3258
Epoch 124/1000
365/36

Epoch 175/1000
365/365 [==============================] - 1s 2ms/step - loss: 36.3027 - I_var: 0.3285 - val_loss: 35.9539 - val_I_var: 0.3375
Epoch 176/1000
365/365 [==============================] - 1s 2ms/step - loss: 35.9308 - I_var: 0.3294 - val_loss: 36.8608 - val_I_var: 0.3111
Epoch 177/1000
365/365 [==============================] - 1s 2ms/step - loss: 36.0767 - I_var: 0.3294 - val_loss: 36.2050 - val_I_var: 0.3301
Epoch 178/1000
365/365 [==============================] - 1s 2ms/step - loss: 36.4031 - I_var: 0.3259 - val_loss: 35.9019 - val_I_var: 0.3389
Epoch 179/1000
365/365 [==============================] - 1s 2ms/step - loss: 36.1702 - I_var: 0.3281 - val_loss: 35.8676 - val_I_var: 0.3400
Epoch 180/1000
365/365 [==============================] - 1s 2ms/step - loss: 36.4742 - I_var: 0.3287 - val_loss: 35.9311 - val_I_var: 0.3377
Epoch 181/1000
281/365 [======================>.......] - ETA: 0s - loss: 35.5274 - I_var: 0.3325

In [ ]:
# Save model
model.save('mpsa_ge_pairwise')

In [ ]:
# Load model
model = mavenn.load('mpsa_ge_pairwise')

In [ ]:
# Get x and y
x_test = test_df['x'].values
y_test = test_df['y'].values

In [ ]:
# Show training history
print('On test data:')

# Compute likelihood information
I_var, dI_var =  model.I_variational(x=x_test, y=y_test)
print(f'I_var_test: {I_var:.3f} +- {dI_var:.3f} bits') 

# Compute predictive information
I_pred, dI_pred = model.I_predictive(x=x_test, y=y_test)
print(f'I_pred_test: {I_pred:.3f} +- {dI_pred:.3f} bits')

I_var_hist = model.history['I_var']
val_I_var_hist = model.history['val_I_var']

fig, ax = plt.subplots(1,1,figsize=[4,4])
ax.plot(I_var_hist, label='I_var_train')
ax.plot(val_I_var_hist, label='I_var_val')
ax.axhline(I_var, color='C2', linestyle=':', label='I_var_test')
ax.axhline(I_pred, color='C3', linestyle=':', label='I_pred_test')
ax.legend()
ax.set_xlabel('epochs')
ax.set_ylabel('bits')
ax.set_title('training hisotry')
ax.set_ylim([0, I_pred*1.2]);

In [ ]:
# Predict latent phentoype values (phi) on test data
phi_test = model.x_to_phi(x_test)

# Predict measurement values (yhat) on test data
yhat_test = model.x_to_yhat(x_test)

# Set phi lims and create grid in phi space
phi_lim = [min(phi_test)-.5, max(phi_test)+.5]
phi_grid = np.linspace(phi_lim[0], phi_lim[1], 1000)

# Compute yhat each phi gridpoint
yhat_grid = model.phi_to_yhat(phi_grid)

# Compute 90% CI for each yhat
q = [0.05, 0.95] #[0.16, 0.84]
yqs_grid = model.yhat_to_yq(yhat_grid, q=q)

# Create figure
fig, ax = plt.subplots(1, 1, figsize=[4, 4])

# Illustrate measurement process with GE curve
ax.scatter(phi_test, y_test, color='C0', s=5, alpha=.2, label='test data')
ax.plot(phi_grid, yhat_grid, linewidth=2, color='C1',
        label='$\hat{y} = g(\phi)$')
ax.plot(phi_grid, yqs_grid[:, 0], linestyle='--', color='C1', label='68% CI')
ax.plot(phi_grid, yqs_grid[:, 1], linestyle='--', color='C1')
ax.set_xlim(phi_lim)
ax.set_xlabel('latent phenotype ($\phi$)')
ax.set_ylabel('measurement ($y$)')
ax.set_title('measurement process')
ax.legend()

# Fix up plot
fig.tight_layout()
plt.show()

In [ ]:
# Plot pairwise parameters
theta = model.get_theta()
fig, ax = plt.subplots(1, 1, figsize=[8, 4])
mavenn.heatmap_pairwise(values=theta['theta_lclc'],
                        alphabet=theta['alphabet'],
                        ax=ax);

In [ ]:
# Test simulate_data
sim_df = model.simulate_dataset(N=1000)
sim_df.head()

In [ ]:
# Is clip by value causing a problem?
yhat_fixed = (yhat_grid - model.y_mean)/model.y_std
a, b, s = model.layer_noise_model.compute_params(yhat_fixed)

fig, axs = plt.subplots(1,3,figsize=[15,5])

axs[0].semilogy(yhat_fixed, a)
axs[1].semilogy(yhat_fixed, b)
axs[2].semilogy(yhat_fixed, s)